# Data preparation as per YOLO requirements

In order to use Yolo, we need to perform several pre-processing steps.
Given the training csv, we will need to conevrt the labels for each box from the given format (list in DataFrame) to yolo format (rescale)

Create a .txt file with all annotations for each image. One line per box and a class identifier

Create a blank .txt for the images without boxes.

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm.auto import tqdm

Open & read CSV file with bounding boxes info

In [2]:
df = pd.read_csv('input/train.csv')
df

,image_id,width,height,bbox,source
0,b6ab77fd7,1024,1024,"[834.0, 222.0, 56.0, 36.0]",usask_1
1,b6ab77fd7,1024,1024,"[226.0, 548.0, 130.0, 58.0]",usask_1
2,b6ab77fd7,1024,1024,"[377.0, 504.0, 74.0, 160.0]",usask_1
3,b6ab77fd7,1024,1024,"[834.0, 95.0, 109.0, 107.0]",usask_1
4,b6ab77fd7,1024,1024,"[26.0, 144.0, 124.0, 117.0]",usask_1
...,...,...,...,...,...
147788,5e0747034,1024,1024,"[64.0, 619.0, 84.0, 95.0]",arvalis_2
147789,5e0747034,1024,1024,"[292.0, 549.0, 107.0, 82.0]",arvalis_2
147790,5e0747034,1024,1024,"[134.0, 228.0, 141.0, 71.0]",arvalis_2
147791,5e0747034,1024,1024,"[430.0, 13.0, 184.0, 79.0]",arvalis_2


In [3]:
repl=[]
for i in range(len(df)):
    row = df.iloc[i,3]
    row = row.replace(',', '')
    row = row.replace('[', '')
    row = row.replace(']', '')
    row= row.split()
    row = [round(float(i)) for i in row]
    repl.append(row)
arr = np.array(repl)
df['box_x'] = arr[:,0]
df['box_y'] = arr[:,1]
df['box_w'] = arr[:,2]
df['box_h'] = arr[:,3]
df

,image_id,width,height,bbox,source,box_x,box_y,box_w,box_h
0,b6ab77fd7,1024,1024,"[834.0, 222.0, 56.0, 36.0]",usask_1,834,222,56,36
1,b6ab77fd7,1024,1024,"[226.0, 548.0, 130.0, 58.0]",usask_1,226,548,130,58
2,b6ab77fd7,1024,1024,"[377.0, 504.0, 74.0, 160.0]",usask_1,377,504,74,160
3,b6ab77fd7,1024,1024,"[834.0, 95.0, 109.0, 107.0]",usask_1,834,95,109,107
4,b6ab77fd7,1024,1024,"[26.0, 144.0, 124.0, 117.0]",usask_1,26,144,124,117
...,...,...,...,...,...,...,...,...,...
147788,5e0747034,1024,1024,"[64.0, 619.0, 84.0, 95.0]",arvalis_2,64,619,84,95
147789,5e0747034,1024,1024,"[292.0, 549.0, 107.0, 82.0]",arvalis_2,292,549,107,82
147790,5e0747034,1024,1024,"[134.0, 228.0, 141.0, 71.0]",arvalis_2,134,228,141,71
147791,5e0747034,1024,1024,"[430.0, 13.0, 184.0, 79.0]",arvalis_2,430,13,184,79


Rescale bounding boxes characteristics as per YOLO requirements

In [5]:
df['0'] = 0
df['yolo_x'] = (df.box_x + df.box_w / 2 ) / 1024
df['yolo_y'] = (df.box_y + df.box_h / 2 ) / 1024
df['yolo_w'] = df.box_w  / 1024
df['yolo_h'] = df.box_h  / 1024
df

,image_id,width,height,bbox,source,box_x,box_y,box_w,box_h,0,yolo_x,yolo_y,yolo_w,yolo_h
0,b6ab77fd7,1024,1024,"[834.0, 222.0, 56.0, 36.0]",usask_1,834,222,56,36,0,0.841797,0.234375,0.054688,0.035156
1,b6ab77fd7,1024,1024,"[226.0, 548.0, 130.0, 58.0]",usask_1,226,548,130,58,0,0.284180,0.563477,0.126953,0.056641
2,b6ab77fd7,1024,1024,"[377.0, 504.0, 74.0, 160.0]",usask_1,377,504,74,160,0,0.404297,0.570312,0.072266,0.156250
3,b6ab77fd7,1024,1024,"[834.0, 95.0, 109.0, 107.0]",usask_1,834,95,109,107,0,0.867676,0.145020,0.106445,0.104492
4,b6ab77fd7,1024,1024,"[26.0, 144.0, 124.0, 117.0]",usask_1,26,144,124,117,0,0.085938,0.197754,0.121094,0.114258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147788,5e0747034,1024,1024,"[64.0, 619.0, 84.0, 95.0]",arvalis_2,64,619,84,95,0,0.103516,0.650879,0.082031,0.092773
147789,5e0747034,1024,1024,"[292.0, 549.0, 107.0, 82.0]",arvalis_2,292,549,107,82,0,0.337402,0.576172,0.104492,0.080078
147790,5e0747034,1024,1024,"[134.0, 228.0, 141.0, 71.0]",arvalis_2,134,228,141,71,0,0.199707,0.257324,0.137695,0.069336
147791,5e0747034,1024,1024,"[430.0, 13.0, 184.0, 79.0]",arvalis_2,430,13,184,79,0,0.509766,0.051270,0.179688,0.077148


Writing .txt files

In [6]:
if not os.path.exists("wheat_yolo_train_data"):
    os.makedirs("wheat_yolo_train_data")

In [7]:
folder_location = "wheat_yolo_train_data"
img_ids = np.unique(df.image_id)
for n, im_id in tqdm(enumerate(img_ids), total=len(img_ids)):
    id_df = df[df.image_id==im_id][['0', 'yolo_x', 'yolo_y', 'yolo_w', 'yolo_h']]
    id_df.to_csv(r'{}\{}.txt'.format(folder_location, im_id), header=None, index=None, sep=' ', mode='a')

Writing empty .txt files where bounding boxes were not found

In [8]:
train_dir = 'input/train/images/train'
file_names = os.listdir(train_dir)
no_box_names = np.setdiff1d(np.array(file_names), np.array(df.image_id+'.jpg'))
print('Total q-ty of empty files', len(no_box_names))
print('file names:')
for i in list(no_box_names):
    file_name = "{}/{}.txt".format(folder_location,i)
    print(file_name)
    with open(file_name, 'w') as fp: 
        pass

Total q-ty of empty files 42
file names:
wheat_yolo_train_data/00b5c6764.jpg.txt
wheat_yolo_train_data/0cf7ef43d.jpg.txt
wheat_yolo_train_data/0e7832f0b.jpg.txt
wheat_yolo_train_data/155c440e8.jpg.txt
wheat_yolo_train_data/1a9792bfc.jpg.txt
wheat_yolo_train_data/2d80f9c49.jpg.txt
wheat_yolo_train_data/2e6c333a7.jpg.txt
wheat_yolo_train_data/3116768f0.jpg.txt
wheat_yolo_train_data/39a0e5556.jpg.txt
wheat_yolo_train_data/45fddc611.jpg.txt
wheat_yolo_train_data/46c6a747a.jpg.txt
wheat_yolo_train_data/49e40c7a4.jpg.txt
wheat_yolo_train_data/526d737d1.jpg.txt
wheat_yolo_train_data/581eaf87b.jpg.txt
wheat_yolo_train_data/5901a73fe.jpg.txt
wheat_yolo_train_data/5cbd092f9.jpg.txt
wheat_yolo_train_data/6584c18c0.jpg.txt
wheat_yolo_train_data/65d0b13aa.jpg.txt
wheat_yolo_train_data/69352f3fa.jpg.txt
wheat_yolo_train_data/7e1caf1f7.jpg.txt
wheat_yolo_train_data/83a94fc79.jpg.txt
wheat_yolo_train_data/8425a537b.jpg.txt
wheat_yolo_train_data/8a5cb62db.jpg.txt
wheat_yolo_train_data/8de789df8.jpg.txt

For training data set was splitted onto three sets mannually and placed on my G-Drive for a further training:
* 2000 - training set;
* 422 - validation set;
* 1000 - testing set;
